# 02.2 - Kedro on Databricks, part 2

_Notes: This notebook is supposed to be run locally from VS Code, all with Databricks Connect_

## Kedro and Databricks Connect

**Databricks Connect** is a client library that allows you to run Spark code locally on your machine while connecting to a remote Databricks cluster for computation. It essentially lets you develop and execute Spark applications from your local IDE or notebook environment, but the actual processing happens on the Databricks cluster.

The **Databricks extension for Visual Studio Code** has several interesting features for connecting to Databricks from VS Code and perform actions sach us deploying and running Databricks Asset Bundles, manage clusters, and easily set up **Databricks Connect**.

Therefore, the two are the perfect companion for developing Kedro projects on VS Code, since you can develop on your IDE while using Databricks compute.

Follow the official documentation to

1. [Install the Databricks extension for VS Code](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/vscode-ext/install)
2. [Configure the appropriate cluster](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/vscode-ext/configure)
3. [Install Databricks Connect](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/vscode-ext/databricks-connect)

_Note: `databricks-connect` provides its own `pyspark` top-level module, and [pip doesn't check for conflicting packages](https://github.com/pypa/pip/issues/4625), so make sure you don't have a [conflicting `pyspark` installation](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/databricks-connect/python/troubleshooting#conflicting-pyspark-installations)!_

In [3]:
%%sh
uv pip install ../../rocketfuel

Using Python 3.11.13 environment at: /Users/Merel_Theisen/anaconda3/envs/bootcamp
Resolved 184 packages in 21.11s
Prepared 10 packages in 15.54s
Uninstalled 2 packages in 103ms
Installed 19 packages in 473ms
 + aiobotocore==2.22.0
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.12.12
 + aioitertools==0.12.0
 + aiosignal==1.3.2
 + botocore==1.37.3
 + docopt==0.6.2
 + frozenlist==1.7.0
 + hdfs==2.7.3
 + ipylab==1.0.0
 + jmespath==1.0.1
 + multidict==6.4.4
 + propcache==0.3.2
 + pyspark==3.5.6
 + rocketfuel==0.1 (from file:///Users/Merel_Theisen/Projects/kedro-academy/kedro-databricks-bootcamp/02_databricks/rocketfuel)
 + s3fs==2025.5.1
 - scikit-learn==1.3.0
 + scikit-learn==1.5.2
 - threadpoolctl==2.2.0
 + threadpoolctl==3.6.0
 + yarl==1.20.1


In [4]:
%load_ext kedro.ipython

[06/10/25 16:09:18] INFO     Registered line magic '%reload_kedro'                                   ]8;id=379825;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=552067;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py#61\61]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=481887;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=764894;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Resolved project path as:                                              ]8;id=463746;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=158448;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /Users/Merel_Theisen/Projects/kedro-academy/kedro-databricks-bootcamp/                
                             02_databricks/rocketfuel.                                                             
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=948330;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=854736;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#65\65]8;;\

                    WARNING  No 'mlflow.yml' config file found in environment. Default            ]8;id=616873;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=171416;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#78\78]8;;\
                             configuration will be used. Use ``kedro mlflow init`` command in CLI                  
                             to customize the configuration.                                                       

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=563994;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_mlflow/config/kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=147616;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_mlflow/config/kedro_mlflow_config.py#293\293]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlruns'). It is                             
                             converted to a valid uri:                                                             
                             'file:///Users/Merel_Theisen/Projects/kedro-academy/kedro-d                           
                             atabricks-bootcamp/02_databricks/rocketfuel/mlruns'                                   

[06/10/25 16:09:39] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=442811;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=308263;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

                    INFO     Kedro project rocketfuel                                               ]8;id=698286;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=379678;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=902844;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=78370;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

[06/10/25 16:09:41] INFO     Registered line magic 'run_viz'                                        ]8;id=755159;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=393460;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py#151\151]8;;\

In [6]:
%reload_kedro ../../rocketfuel

[06/10/25 16:09:59] INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=487733;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=92485;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#65\65]8;;\

                    WARNING  No 'mlflow.yml' config file found in environment. Default            ]8;id=823743;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=488311;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#78\78]8;;\
                             configuration will be used. Use ``kedro mlflow init`` command in CLI                  
                             to customize the configuration.                                                       

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=522915;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_mlflow/config/kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=312180;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_mlflow/config/kedro_mlflow_config.py#293\293]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlruns'). It is                             
                             converted to a valid uri:                                                             
                             'file:///Users/Merel_Theisen/Projects/kedro-academy/kedro-d                           
                             atabricks-bootcamp/02_databricks/rocketfuel/mlruns'                                   

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=487890;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=596827;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[06/10/25 16:10:00] INFO     Kedro project rocketfuel                                               ]8;id=865279;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=977051;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=717155;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=576302;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=809523;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=504820;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/ipython/__init__.py#151\151]8;;\

In [7]:
catalog.list()


[
    'companies',
    'reviews',
    'shuttles_excel',
    'shuttles@csv',
    'shuttles@spark',
    'preprocessed_companies',
    'preprocessed_shuttles',
    'preprocessed_reviews',
    'model_input_table@spark',
    'model_input_table@pandas',
    'regressor',
    'parameters',
    'params:model_options',
    'params:model_options.test_size',
    'params:model_options.random_state',
    'params:model_options.features'
]

Notice how data is loaded as a PySpark DataFrame, directly from Databricks Unity Catalog!

In [8]:
catalog._get_dataset("companies")

kedro_datasets.spark.spark_dataset.SparkDataset(filepath='/Users/Merel_Theisen/Projects/kedro-academy/kedro-databricks-bootcamp/02_databricks/rocketfuel/data/01_raw/companies.csv', file_format='csv', load_args={'header': True, 'inferSchema': True}, save_args={'sep': ',', 'header': True, 'mode': 'overwrite'})

In [9]:
display(catalog.load("companies"))

[06/10/25 16:10:16] INFO     Loading data from companies (SparkDataset)...                      ]8;id=803164;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=712935;file:///Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro/io/core.py:245   │
│ in load                                                                                          │
│                                                                                                  │
│   242 │   │   │   self._logger.debug("Loading %s", str(self))                                    │
│   243 │   │   │                                                                                  │
│   244 │   │   │   try:                                                                           │
│ ❱ 245 │   │   │   │   return load_func(self)                                                     │
│   246 │   │   │   except DatasetError:                                                           │
│   247 │   │   │   │   raise                                                                      │
│   248 │   │   │   except Exception as exc:                                                       │
│                                                                                                  │
│ /Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_datasets/spark/s │
│ park_dataset.py:310 in load                                                                      │
│                                                                                                  │
│   307 │                                                                                          │
│   308 │   def load(self) -> DataFrame:                                                           │
│   309 │   │   load_path = strip_dbfs_prefix(self._fs_prefix + str(self._get_load_path()))        │
│ ❱ 310 │   │   read_obj = get_spark().read                                                        │
│   311 │   │                                                                                      │
│   312 │   │   # Pass schema if defined                                                           │
│   313 │   │   if self._schema:                                                                   │
│                                                                                                  │
│ /Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/kedro_datasets/_utils/ │
│ spark_utils.py:22 in get_spark                                                                   │
│                                                                                                  │
│   19 │   │   from databricks.connect import DatabricksSession                                    │
│   20 │   │                                                                                       │
│   21 │   │   # We can't test this as there's no Databricks test env available                    │
│ ❱ 22 │   │   spark = DatabricksSession.builder.getOrCreate()  # pragma: no cover                 │
│   23 │                                                                                           │
│   24 │   except ImportError:                                                                     │
│   25 │   │   # For "normal" spark sessions that don't use databricks-connect                     │
│                                                                                                  │
│ /Users/Merel_Theisen/anaconda3/envs/bootcamp/lib/python3.11/site-packages/databricks/connect/ses │
│ sion.py:452 in getOrCreate                                                                       │
│                                                                                                  │
│   449 │   │   │   # use the default values that may be supplied from the SDK Config              │
│   450 │   │   │   logger.debug("Falling back to default configuration from the SDK.")            │
│   451 │   │   │   config = Config()                        

## Namespaces for pipeline grouping

Namespaces allow you to group nodes, ensuring clear dependencies and separation within a pipeline while maintaining a consistent structure. Like with pipelines or tags, you can enable selective execution using namespaces, and you cannot run more than one namespace simultaneously — Kedro allows executing one namespace at a time.

Defining namespace at Pipeline-level: When applying a namespace at the pipeline level, Kedro automatically renames all inputs, outputs, and parameters within that pipeline. You will need to update your catalog accordingly. If you don’t want to change the names of your inputs, outputs, or parameters with the `namespace_name`. prefix while using a namespace, you should list these objects inside the corresponding parameters of the `pipeline()` creation function. For example:

```python
return pipeline(
    base_pipeline,
    namespace = "new_namespaced_pipeline", # With that namespace, "new_namespaced_pipeline" prefix will be added to inputs, outputs, params, and node names
    inputs={"the_original_input_name"}, # Inputs remain the same, without namespace prefix
)
```

Namespaces allow you to group your nodes and pipelines more efficiently in deployment, for example, when running your pipeline as a Databricks Job or Asset Bundle. 

To add a namespace to a Kedro pipeline, you can use the `namespace` argument in the `pipeline()` function. This argument accepts a string that will be used as a prefix for all nodes, inputs, outputs, and parameters within that pipeline. Note that you'll have to update your catalog accordingly, as Kedro expects that all inputs, outputs, and parameters within that pipeline include the namespace prefix. 

For example, to add a namespace to our `data_science` pipeline you'll have to modifify the code to:

In [ ]:
# data_science/pipeline.py

def create_pipeline(**kwargs) -> Pipeline:
    return pipeline(
        [
            node(
                func=split_data,
                inputs=["model_input_table@pandas", "params:model_options"],
                outputs=["X_train", "X_test", "y_train", "y_test"],
                name="split_data_node",
            ),
            node(
                func=train_model,
                inputs=["X_train", "y_train"],
                outputs="regressor",
                name="train_model_node",
            ),
            node(
                func=evaluate_model,
                inputs=["regressor", "X_test", "y_test"],
                outputs=None,
                name="evaluate_model_node",
            ),
        ], namespace="ds"
    )


### Update catalog.yml to add namespace prefixes to relevant datasets

```yaml
ds.model_input_table@pandas:
  type: pandas.ParquetDataset
  filepath: data/03_primary/model_input_table.parquet
```

### Update parameters.yml to add namespace prefixes to relevant parameters

```yaml
ds.model_options:
  test_size: 0.2
  random_state: 3
  features:
    - engines
    - passenger_capacity
    - crew
    - d_check_complete
    - moon_clearance_complete
    - iata_approved
    - company_rating
    - review_scores_rating
```

## Integration with Databricks MLflow

### Log Kedro runs as MLflow experiments

There are 2 types of MLflow experiments in Databricks:
- **Workspace** experiments are not associated with any notebook, and any notebook can log a run to these experiments by using the experiment ID or the experiment name. _They cannot be created inside Git folders._
- **Notebook** experiments are associated with a specific notebook. _They are note checked into source control_.

Therefore, for personal experimentation **notebook** experiments are more appropriate, and for collaboration **workspace** experiments can be created in a regular workspace folder outside of Git.

In [ ]:
%%sh
uv pip install ./rocketfuel

In [ ]:
%load_ext kedro.ipython

Kedro runs can be logged in MLflow using the [`kedro-mlflow`](https://kedro-mlflow.readthedocs.io/) community plugin.

In [7]:
%%sh
uv pip install kedro-mlflow

[06/09/25 16:54:11] WARNING  /Users/juan_cano/.ipython/profile_default/startup/00-databricks-init-c ]8;id=523748;file:///opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/warnings.py\warnings.py]8;;\:]8;id=81443;file:///opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/warnings.py#112\112]8;;\
                             db6174794c74fbc909de1ce43bfe286.py:341: UserWarning:                                  
                             %%sh is not supported on Databricks. This notebook might fail when                    
                             running on a Databricks cluster.                                                      
                             Consider using %sh instead.                                                           
                               warnings.warn(                                                                      
                                                                                                                   

Using Python 3.12.9 environment at: /Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv
Audited 1 package in 979ms


`kedro-mlflow` can take [configuration](https://kedro-mlflow.readthedocs.io/en/0.14.4/source/03_experiment_tracking/01_experiment_tracking/01_configuration.html) from `conf/<environment>/mlflow.yml`, which can be used to configure the experiment name:

- For **notebook** experiments, you have to configure the experiment name to match the full path of the notebook.
- For **workspace** experiments, the experiment name would be the full path to the experiments folder in the workspace.

To this end, let's add some OmegaConf syntax to `mlflow.yml` so that the experiment name can be specified from the outside, while taking a default value if not present:

In [ ]:
%%writefile rocketfuel/conf/databricks/mlflow.yml
tracking:
  experiment:
    name: ${runtime_params:mlflow_experiment_name, ${kedro_root:}}

Let's try to set up a **notebook** experiment. For this, extract the notebook path:

In [ ]:
notebook_path = dbutils.entry_point.getDbutils().notebook().getContext().notebookPath().get()
notebook_path

And pass that as a runtime parameter to specify the experiment name:

_Note: Extra params cannot contain spaces when passed to `%reload_kedro`, see [this issue](https://github.com/kedro-org/kedro/issues/4813)_

In [ ]:
%reload_kedro rocketfuel --env databricks --params mlflow_experiment_name=$notebook_path

Now, every time a Kedro pipeline is run, it's logged as al MLflow run:

In [ ]:
session.run("load_data")

![MLflow run corresponding to a Kedro run on Databricks](./kedro-databricks-mlflow-run.png)

### Log artifacts as MLflow models in the Unity Catalog

In [ ]:
%sql
SHOW CATALOGS;

In [ ]:
# Test code
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier

import mlflow

# Train a sklearn model on the iris dataset
X, y = datasets.load_iris(return_X_y=True, as_frame=True)
clf = RandomForestClassifier(max_depth=7)
clf.fit(X, y)

# Note that the UC model name follows the pattern
# <catalog_name>.<schema_name>.<model_name>, corresponding to
# the catalog, schema, and registered model name
# in Unity Catalog under which to create the version
# The registered model will be created if it doesn't already exist
autolog_run = mlflow.last_active_run()
model_uri = "runs:/{}/model".format(autolog_run.info.run_id)  # NOTE: Can this be automatic?
mlflow.register_model(model_uri, "aza-databricks-b9b7aae-catalog.rocketfuel.iris_model")  # NOTE: Can this be automatic?

In [ ]:
session.run("__default__")

### Register models using the Databricks Unity Catalog

_**Note**: fsspec uses the DBFS API, which is not compatible with Unity Catalog according to https://github.com/fsspec/filesystem_spec/issues/1656_